The aim is to compare questions to see if they are asking the same thing

In [ ]:
import tensorflow_hub as hub

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, TextVectorization,Dropout, Embedding,GlobalMaxPooling1D ,Conv1D,LSTM, TextVectorization, Input
from tensorflow.keras.regularizers import l2
import pandas as pd
import tensorflow as tf
import pandas as pd


In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  from sklearn.metrics import accuracy_score, precision_recall_fscore_support
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
#creating data
data_questions_y = data[ ["question1", "question2", "is_duplicate"]]

In [ ]:
data_questions_y.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
X = data_questions_y.drop("is_duplicate", axis =1)
y = data_questions_y.is_duplicate

In [ ]:
import plotly.graph_objects as go

In [ ]:
#Creating a pie plot 
def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    
    pie_plot = go.Pie(labels=labels, values=counts, hole=.3)
    fig = go.Figure(data=[pie_plot])
    fig.update_layout(title_text='Distribution for %s' % pd_series.name)
    fig.show()

In [ ]:
plot_distribution(data.is_duplicate)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#creating split
X_train, X_test, y_train, y_test = train_test_split(X.astype("str"), y, test_size=0.1, random_state=42)

In [ ]:
#scoring function 
def model_class_scores(pred, actual):
    """"
    calculates accuracy, precision, recall and f1 score, binary classification 

    """

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    scores = {"accuracy" : accuracy_score(actual, pred),
    "precision" :   precision_score(actual, pred),
    "recall" : recall_score(actual, pred),
    "f1" : f1_score(actual, pred)}
     
    return scores


In [ ]:
#binary tensorflow prediction function
def tensorflow_predict(model, data):
  import tensorflow as tf
  prediction = model.predict(data)
  pred = tf.squeeze(tf.round(prediction))
  return pred

# Creating Model

In [ ]:
#creating text vectorizer 
text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_mode = "int",
                                    output_sequence_length = 25)


In [ ]:
#list of all questions 
all_questions = pd.concat([X_train.question1, X_train.question2])

In [ ]:
#fitting text vectorizer 
all_questions.to_numpy()
text_vectorizer.adapt(all_questions.to_numpy())

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential, Model

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, concatenate, Conv1D, MaxPooling1D, Bidirectional

In [ ]:
#Both pairs compared of comments are fed into the Neural Net
#An embedding is created for each net 
#Both comments are then put through a LSTM 
#their outputs are concatenated 
#Then the output is run through a feedforward net.

def build_model():

    
    input_1 = Input(shape= (1,),  dtype= tf.string) #input of question 1
    input_2 = Input(shape= (1, ),  dtype = tf.string) #input question 2
    
    sentence_1 = text_vectorizer(input_1)
    sentence_2 = text_vectorizer(input_2)

    sentence_1L1 = Embedding(10000, 512)(sentence_1)
    sentence_2L1 =  Embedding(10000, 512)(sentence_2)
    
    sentence_1L2 = LSTM(64, return_sequences = False, dropout = 0.5)(sentence_1L1)
    sentence_2L2 = LSTM(64, return_sequences = False, dropout =0.5)(sentence_2L1)
    
    output = concatenate([sentence_1L2, sentence_1L2])
    output = Dense(32, activation = 'relu')(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2],
        outputs = [output]
    
    
    
    )
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [ ]:
rnn = build_model()

In [ ]:
rnn.fit([X_train.question1, X_train.question2],y_train, epochs = 10, batch_size = 1000, validation_data=([X_test.question1, X_test.question2], y_test))

Epoch 1/10
364/364 [==============================] - 11s 18ms/step - loss: 0.5441 - accuracy: 0.7251 - val_loss: 0.5128 - val_accuracy: 0.7463
Epoch 2/10
364/364 [==============================] - 6s 17ms/step - loss: 0.5007 - accuracy: 0.7540 - val_loss: 0.5042 - val_accuracy: 0.7514
Epoch 3/10
364/364 [==============================] - 6s 17ms/step - loss: 0.4836 - accuracy: 0.7636 - val_loss: 0.5018 - val_accuracy: 0.7548
Epoch 4/10
364/364 [==============================] - 6s 17ms/step - loss: 0.4687 - accuracy: 0.7724 - val_loss: 0.5033 - val_accuracy: 0.7588
Epoch 5/10
364/364 [==============================] - 6s 17ms/step - loss: 0.4533 - accuracy: 0.7807 - val_loss: 0.5008 - val_accuracy: 0.7607
Epoch 6/10
364/364 [==============================] - 6s 17ms/step - loss: 0.4380 - accuracy: 0.7891 - val_loss: 0.5025 - val_accuracy: 0.7596
Epoch 7/10
364/364 [==============================] - 6s 17ms/step - loss: 0.4228 - accuracy: 0.7975 - val_loss: 0.5042 - val_accuracy: 0.761

In [ ]:
rnn_pred = tensorflow_predict(rnn, [X_test.question1, X_test.question2])

In [ ]:
rnn_score = model_class_scores(rnn_pred, y_test)
rnn_score

{'accuracy': 0.7627198298251255,
 'f1': 0.6562757533412161,
 'precision': 0.705710102489019,
 'recall': 0.6133136887222074}

In [ ]:
from tensorflow.keras.layers import Bidirectional, GlobalAveragePooling1D, Dropout, MaxPooling1D, concatenate

In [ ]:
#Both pairs compared of comments are fed into the Neural Net
#An embedding is created for each net 

#Both comments are then put through a 1DCov
#Both comments are then put through a Bidirectional LSTM 
#their outputs are concatenated 
#Then the output is run through a feedforward net.


def build_model():

    
    input_1 = Input(shape= (1,),  dtype= tf.string)
    input_2 = Input(shape= (1, ),  dtype = tf.string)
    
    sentence_1 = text_vectorizer(input_1)
    sentence_2 = text_vectorizer(input_2)

    
    sentence_1L1 = Embedding(10000, 512)(sentence_1)
    sentence_2L1 =  Embedding(10000, 512)(sentence_2)
    
  
    
    
    sentence_1L4 = Bidirectional(LSTM(64, return_sequences = False, dropout =0.4))(sentence_1L1)
    sentence_2L4 = Bidirectional(LSTM(64, return_sequences = False, dropout = 0.4))(sentence_2L1)
    Sentence_dif = sentence_2L4 - sentence_1L4 #Minusing the outputs
    
    output = concatenate([Sentence_dif,  sentence_1L4,sentence_2L4])
    output = Dense(64, activation = 'relu')(output)
    output= Dropout(0.1)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2],
        outputs = output
    
    
    
    )
    
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [ ]:
CLSTM_model = build_model()

In [ ]:
CLSTM_model.fit([[X_train.question1, X_train.question2]],y_train, epochs = 10, batch_size = 1500, validation_data=([X_test.question1, X_test.question2], y_test))

Epoch 1/10
243/243 [==============================] - 23s 73ms/step - loss: 0.5170 - accuracy: 0.7453 - val_loss: 0.4876 - val_accuracy: 0.7630
Epoch 2/10
243/243 [==============================] - 16s 67ms/step - loss: 0.4631 - accuracy: 0.7770 - val_loss: 0.4621 - val_accuracy: 0.7760
Epoch 3/10
243/243 [==============================] - 16s 68ms/step - loss: 0.4299 - accuracy: 0.7950 - val_loss: 0.4543 - val_accuracy: 0.7804
Epoch 4/10
243/243 [==============================] - 16s 67ms/step - loss: 0.4030 - accuracy: 0.8101 - val_loss: 0.4436 - val_accuracy: 0.7908
Epoch 5/10
243/243 [==============================] - 16s 67ms/step - loss: 0.3799 - accuracy: 0.8238 - val_loss: 0.4469 - val_accuracy: 0.7909
Epoch 6/10
243/243 [==============================] - 16s 67ms/step - loss: 0.3597 - accuracy: 0.8351 - val_loss: 0.4483 - val_accuracy: 0.7884
Epoch 7/10
243/243 [==============================] - 16s 67ms/step - loss: 0.3400 - accuracy: 0.8461 - val_loss: 0.4597 - val_accuracy:

In [ ]:
CLSTM_pred = tensorflow_predict(CLSTM_model, [X_test.question1, X_test.question2])

In [ ]:
CLSTM_score = model_class_scores(CLSTM_pred, y_test)
CLSTM_score

{'accuracy': 0.8011575849019268,
 'f1': 0.719416425255663,
 'precision': 0.7512209344704425,
 'recall': 0.6901955531743905}

#Universal Sentence encoder 

In [ ]:
import tensorflow_hub as hub

In [ ]:
#loading the sentence encoder 
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", 
                                        input_shape =[],
                                        dtype = tf.string,
                                        name = "badboy")

In [ ]:
#creating sentence encoder layer
trained_emb_model = Sequential([sentence_encoder_layer,
                                ])

In [ ]:

def build_model():

    
    input_1 = Input(shape= (),  dtype= tf.string)
    input_2 = Input(shape= (),  dtype = tf.string)
    
    sentence_1 = trained_emb_model(input_1)
    sentence_2 = trained_emb_model(input_2)

    
    output = sentence_1 - sentence_2 #difference between sentence embedding 
    output = Dense(64, activation = 'relu')(output)
    output= Dropout(0.1)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2],
        outputs = output
    
    )
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model
    

In [ ]:
emb_model = build_model()

In [ ]:
emb_model.fit([X_train.question1, X_train.question2],y_train, epochs = 20, batch_size = 1000, validation_data=([X_test.question1, X_test.question2], y_test))

Epoch 1/20
364/364 [==============================] - 55s 136ms/step - loss: 0.5956 - accuracy: 0.6397 - val_loss: 0.5466 - val_accuracy: 0.6822
Epoch 2/20
364/364 [==============================] - 51s 140ms/step - loss: 0.5165 - accuracy: 0.7194 - val_loss: 0.4948 - val_accuracy: 0.7420
Epoch 3/20
364/364 [==============================] - 56s 153ms/step - loss: 0.4751 - accuracy: 0.7591 - val_loss: 0.4629 - val_accuracy: 0.7648
Epoch 4/20
364/364 [==============================] - 45s 123ms/step - loss: 0.4501 - accuracy: 0.7739 - val_loss: 0.4478 - val_accuracy: 0.7741
Epoch 5/20
364/364 [==============================] - 43s 117ms/step - loss: 0.4390 - accuracy: 0.7804 - val_loss: 0.4404 - val_accuracy: 0.7803
Epoch 6/20
364/364 [==============================] - 43s 119ms/step - loss: 0.4315 - accuracy: 0.7855 - val_loss: 0.4345 - val_accuracy: 0.7835
Epoch 7/20
364/364 [==============================] - 43s 119ms/step - loss: 0.4251 - accuracy: 0.7893 - val_loss: 0.4306 - val_ac

In [ ]:
#buildeing model
def build_model():

    
    input_1 = Input(shape= (),  dtype= tf.string)
    input_2 = Input(shape= (),  dtype = tf.string)
    
    sentence_1 = trained_emb_model(input_1)
    sentence_2 = trained_emb_model(input_2)

    
    diff = sentence_1 - sentence_2
    output = concatenate([diff,  sentence_1,sentence_2])
    output = Dense(64, activation = 'relu')(output)
    output= Dropout(0.1)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2],
        outputs = output
    
    )
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [ ]:
emb_model_diff = build_model()

In [ ]:
emb_model_diff.fit([X_train.question1, X_train.question2],y_train, epochs = 30, batch_size = 2048, validation_data=([X_test.question1, X_test.question2], y_test))

Epoch 1/30
178/178 [==============================] - 37s 199ms/step - loss: 0.5652 - accuracy: 0.7147 - val_loss: 0.5219 - val_accuracy: 0.7401
Epoch 2/30
178/178 [==============================] - 35s 198ms/step - loss: 0.4904 - accuracy: 0.7593 - val_loss: 0.4684 - val_accuracy: 0.7707
Epoch 3/30
178/178 [==============================] - 35s 197ms/step - loss: 0.4499 - accuracy: 0.7825 - val_loss: 0.4376 - val_accuracy: 0.7889
Epoch 4/30
178/178 [==============================] - 35s 197ms/step - loss: 0.4235 - accuracy: 0.7979 - val_loss: 0.4152 - val_accuracy: 0.8025
Epoch 5/30
178/178 [==============================] - 35s 198ms/step - loss: 0.4041 - accuracy: 0.8097 - val_loss: 0.3999 - val_accuracy: 0.8126
Epoch 6/30
178/178 [==============================] - 35s 197ms/step - loss: 0.3891 - accuracy: 0.8182 - val_loss: 0.3886 - val_accuracy: 0.8174
Epoch 7/30
178/178 [==============================] - 35s 199ms/step - loss: 0.3780 - accuracy: 0.8241 - val_loss: 0.3806 - val_ac

In [ ]:
def build_model():

    
    input_1 = Input(shape= (),  dtype= tf.string)
    input_2 = Input(shape= (),  dtype = tf.string)
    
    sentence_1 = trained_emb_model(input_1)
    sentence_2 = trained_emb_model(input_2)

    
    diff = sentence_1 - sentence_2
    output = concatenate([diff,  sentence_1,sentence_2])
    output = Dense(128, activation = 'relu')(output)
    output= Dropout(0.15)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2],
        outputs = output
    
    )
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model
    

In [ ]:
emb_model_diff_more= build_model()

In [ ]:
emb_model_diff_more.fit([X_train.question1, X_train.question2],y_train, epochs = 15, batch_size = 2048, validation_data=([X_test.question1, X_test.question2], y_test))

Epoch 1/15
178/178 [==============================] - 45s 227ms/step - loss: 0.5387 - accuracy: 0.7314 - val_loss: 0.4812 - val_accuracy: 0.7669
Epoch 2/15
178/178 [==============================] - 39s 218ms/step - loss: 0.4509 - accuracy: 0.7816 - val_loss: 0.4327 - val_accuracy: 0.7917
Epoch 3/15
178/178 [==============================] - 35s 197ms/step - loss: 0.4112 - accuracy: 0.8052 - val_loss: 0.4001 - val_accuracy: 0.8122
Epoch 4/15
178/178 [==============================] - 34s 194ms/step - loss: 0.3863 - accuracy: 0.8192 - val_loss: 0.3826 - val_accuracy: 0.8199
Epoch 5/15
178/178 [==============================] - 35s 195ms/step - loss: 0.3697 - accuracy: 0.8293 - val_loss: 0.3722 - val_accuracy: 0.8281
Epoch 6/15
178/178 [==============================] - 37s 209ms/step - loss: 0.3570 - accuracy: 0.8362 - val_loss: 0.3652 - val_accuracy: 0.8289
Epoch 7/15
178/178 [==============================] - 42s 237ms/step - loss: 0.3472 - accuracy: 0.8416 - val_loss: 0.3579 - val_ac

In [ ]:
pred = tensorflow_predict(emb_model_diff_more, [X_test.question1, X_test.question2])

In [ ]:
model_class_scores(pred, y_test)

{'accuracy': 0.8454574686487423,
 'f1': 0.7899267029789524,
 'precision': 0.7931802835921674,
 'recall': 0.7866997053308331}

#Character embedding 

In [ ]:
def split_cha(text):
  return " ".join(list(text))
import pandas as pd  

In [ ]:
#spliting data texts into characters 
question1_char = pd.Series([split_cha(text) for text in X_train.question1])
question2_char = pd.Series([split_cha(text) for text in X_train.question2])
test_question1_char = pd.Series([split_cha(text) for text in X_test.question1])
test_question2_char = pd.Series([split_cha(text) for text in X_test.question2])

In [ ]:
all_questions_char = pd.concat([question1_char, question2_char])

In [ ]:
#95 quantile character length 
np.quantile([len(sen.split(" ")) for sen in all_questions_char], 0.95)

146.0

In [ ]:
#character vectorizer
char_vectoriser = TextVectorization(max_tokens=70,
                                    output_mode = "int",
                                    output_sequence_length = 146,


                                    )

In [ ]:
#fitting chracter vec
char_vectoriser.adapt(all_questions_char)

In [ ]:
def build_model():

    
    input_1 = Input(shape= (),  dtype= tf.string)
    input_2 = Input(shape= (),  dtype = tf.string)
    input_char_1 = Input(shape= (1,),  dtype = tf.string)
    input_char_2 = Input(shape= (1,),  dtype = tf.string)


    sentence_1 = trained_emb_model(input_1)
    sentence_2 = trained_emb_model(input_2)

    #charter 1
    ctext1 = char_vectoriser(input_char_1)
    ctext1 = Embedding(28,512)(ctext1)
    ctext1 = Conv1D(64,5,activation = "relu")(ctext1)
    ctext1 = GlobalMaxPooling1D()(ctext1)
    
    #character 2
    ctext2 = char_vectoriser(input_char_2)
    ctext2 = Embedding(28,512)(ctext2)
    ctext2 = Conv1D(64,5,activation = "relu")(ctext2)
    ctext2 = GlobalMaxPooling1D()(ctext2)

    char_diff =  ctext1 - ctext2 
    diff = sentence_1 - sentence_2
    output = concatenate([diff,  sentence_1,sentence_2, ctext1, ctext2, char_diff])
    output = Dense(128, activation = 'relu')(output)
    output= Dropout(0.15)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2, input_char_1, input_char_2],
        outputs = output
    
    )
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [ ]:
charem_model = build_model()

In [ ]:
charem_model.fit([X_train.question1, X_train.question2, question1_char, question2_char ],
    y_train,
    epochs = 15, 
    batch_size = 2048, 
    validation_data=([X_test.question1, X_test.question2, test_question1_char, test_question2_char  ], y_test))

Epoch 1/15
178/178 [==============================] - 75s 368ms/step - loss: 0.5489 - accuracy: 0.7223 - val_loss: 0.5158 - val_accuracy: 0.7289
Epoch 2/15
178/178 [==============================] - 66s 372ms/step - loss: 0.4672 - accuracy: 0.7714 - val_loss: 0.4463 - val_accuracy: 0.7874
Epoch 3/15
178/178 [==============================] - 66s 370ms/step - loss: 0.4270 - accuracy: 0.7953 - val_loss: 0.4124 - val_accuracy: 0.8048
Epoch 4/15
178/178 [==============================] - 66s 368ms/step - loss: 0.3999 - accuracy: 0.8111 - val_loss: 0.3887 - val_accuracy: 0.8163
Epoch 5/15
178/178 [==============================] - 66s 369ms/step - loss: 0.3791 - accuracy: 0.8240 - val_loss: 0.3766 - val_accuracy: 0.8242
Epoch 6/15
178/178 [==============================] - 66s 370ms/step - loss: 0.3630 - accuracy: 0.8333 - val_loss: 0.3646 - val_accuracy: 0.8300
Epoch 7/15
178/178 [==============================] - 66s 369ms/step - loss: 0.3491 - accuracy: 0.8404 - val_loss: 0.3872 - val_ac

In [ ]:
pred = tensorflow_predict(charem_model, [X_test.question1, X_test.question2, test_question1_char, test_question2_char ])

In [ ]:
model_class_scores(pred, y_test)

{'accuracy': 0.8477083281802666,
 'f1': 0.7970331300477995,
 'precision': 0.784847107706291,
 'recall': 0.8096035360300027}

Character embedding LSTM

In [ ]:
sentence_1L4 = Bidirectional(LSTM(64, return_sequences = False, dropout =0.4))(sentence_1L1)

In [ ]:
def build_model():

    
    input_1 = Input(shape= (),  dtype= tf.string)
    input_2 = Input(shape= (),  dtype = tf.string)
    input_char_1 = Input(shape= (1,),  dtype = tf.string)
    input_char_2 = Input(shape= (1,),  dtype = tf.string)


    sentence_1 = trained_emb_model(input_1)
    sentence_2 = trained_emb_model(input_2)

    #charter 1
    ctext1 = char_vectoriser(input_char_1)
    ctext1 = Embedding(28,512)(ctext1)
    ctext1 = Bidirectional(LSTM(64, return_sequences = False, dropout =0.4))(ctext1)
    
    #character 2
    ctext2 = char_vectoriser(input_char_2)
    ctext2 = Embedding(28,512)(ctext2)
    ctext2 = Bidirectional(LSTM(64, return_sequences = False, dropout =0.4))(ctext2)
  

    char_diff =  ctext1 - ctext2 
    diff = sentence_1 - sentence_2
    output = concatenate([diff,  sentence_1,sentence_2, ctext1, ctext2, char_diff])
    output = Dense(128, activation = 'relu')(output)
    output= Dropout(0.15)(output)
    output = Dense(1, activation = 'sigmoid')(output)
    
    
    model = Model(
    
        inputs = [input_1, input_2, input_char_1, input_char_2],
        outputs = output
    
    )
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [ ]:
bichar_model = build_model()

In [ ]:
bichar_model.fit([X_train.question1, X_train.question2, question1_char, question2_char ],
    y_train,
    epochs = 15, 
    batch_size = 2048, 
    validation_data=([X_test.question1, X_test.question2, test_question1_char, test_question2_char  ], y_test))

Epoch 1/15
178/178 [==============================] - 110s 575ms/step - loss: 0.5537 - accuracy: 0.7193 - val_loss: 0.5006 - val_accuracy: 0.7574
Epoch 2/15
178/178 [==============================] - 98s 549ms/step - loss: 0.4716 - accuracy: 0.7701 - val_loss: 0.4462 - val_accuracy: 0.7859
Epoch 3/15
178/178 [==============================] - 97s 547ms/step - loss: 0.4268 - accuracy: 0.7966 - val_loss: 0.4125 - val_accuracy: 0.8081
Epoch 4/15
178/178 [==============================] - 98s 549ms/step - loss: 0.3998 - accuracy: 0.8132 - val_loss: 0.3916 - val_accuracy: 0.8182
Epoch 5/15
178/178 [==============================] - 97s 545ms/step - loss: 0.3782 - accuracy: 0.8249 - val_loss: 0.3781 - val_accuracy: 0.8228
Epoch 6/15
178/178 [==============================] - 98s 549ms/step - loss: 0.3629 - accuracy: 0.8331 - val_loss: 0.3677 - val_accuracy: 0.8301
Epoch 7/15
178/178 [==============================] - 98s 550ms/step - loss: 0.3518 - accuracy: 0.8391 - val_loss: 0.3602 - val_a

In [ ]:
bichar_model.fit([X_train.question1, X_train.question2, question1_char, question2_char ],
    y_train,
    epochs = 5, 
    batch_size = 2048, 
    validation_data=([X_test.question1, X_test.question2, test_question1_char, test_question2_char  ], y_test))

Epoch 1/5
178/178 [==============================] - 98s 550ms/step - loss: 0.2911 - accuracy: 0.8724 - val_loss: 0.3397 - val_accuracy: 0.8472
Epoch 2/5
178/178 [==============================] - 98s 551ms/step - loss: 0.2864 - accuracy: 0.8747 - val_loss: 0.3408 - val_accuracy: 0.8469
Epoch 3/5
178/178 [==============================] - 98s 551ms/step - loss: 0.2809 - accuracy: 0.8773 - val_loss: 0.3377 - val_accuracy: 0.8485
Epoch 4/5
178/178 [==============================] - 99s 558ms/step - loss: 0.2771 - accuracy: 0.8792 - val_loss: 0.3374 - val_accuracy: 0.8494
Epoch 5/5
178/178 [==============================] - 97s 547ms/step - loss: 0.2724 - accuracy: 0.8816 - val_loss: 0.3416 - val_accuracy: 0.8463


In [ ]:
pred = tensorflow_predict(bichar_model, [X_test.question1, X_test.question2, test_question1_char, test_question2_char ])
model_class_scores(pred, y_test)

{'accuracy': 0.8463479185733014,
 'f1': 0.7841406630064632,
 'precision': 0.8148923876931966,
 'recall': 0.755625502276989}